##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Sequential モデル

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/keras/sequential_model"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で実行</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/keras/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">	Google Colabで実行</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/keras/sequential_model.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/keras/sequential_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a> </td>
</table>

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Sequential モデルの使用が適している場合

`Sequential`モデルは、各レイヤーに** 1 つの入力テンソルと 1 つの出力テンソルのみ**がある**レイヤーのプレーンスタック**に適しています。

概略的には、以下の`Sequential`モデルを参照してください。

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

上記は次の関数と同じです。

In [ ]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

以下の場合、Sequential モデルは**適切ではありません**。

- モデルに複数の入力または複数の出力がある場合
- レイヤーに複数の入力または複数の出力がある場合
- レイヤーの共有を行う必要がある場合
- 非線形トポロジーが必要な場合（残差を用いた接続、複数の分岐点をもつモデルなど）

## Sequential モデルの作成

レイヤーのリストを Sequential コンストラクタに渡すことにより、Sequential モデルを作成できます。

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

そのレイヤーには、`layers`属性を介してアクセスできます。

In [ ]:
model.layers

また、`add()`メソッドを使用して Sequential モデルを作成することもできます。

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

また、レイヤーを削除するには、 `pop()` メソッドが対応します。Sequential モデルは、レイヤーのリストのように動作します。

In [ ]:
model.pop()
print(len(model.layers))  # 2

また、Sequential コンストラクタは、Keras のレイヤーやモデルと同様に、`name`引数を受け入れます。これは、セマンティックに意味のある命名で TensorBoard グラフに注釈を付けるのに役立ちます。

In [ ]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

## 事前に入力形状を指定

一般的に、Keras のすべてのレイヤーは、重みを作成できるようにするために、入力の形状を知る必要があります。このようなレイヤーを作成する場合、当初は重みがありません。

In [ ]:
layer = layers.Dense(3)
layer.weights  # Empty

重みの形状は入力の形状に依存するため、入力時に最初に呼び出されたときに重みが作成されます。

In [ ]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

これは Sequential モデルでも同じです。入力形状なしで Sequential モデルをインスタンス化すると、重みがないために「構築」されません (`model.weights`を呼び出すと、これを示すエラーが発生します)。重みは、モデルが最初に入力データを確認したときに作成されます。

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

モデルが「構築」されたら、その`summary()`メソッドを呼び出して、その内容を表示できます。

In [ ]:
model.summary()

ただし、Sequential モデルを段階的に構築する場合、その時点の出力形状を含め、それまでのモデルの概要を表示できると非常に便利です。 この場合、モデルに`Input`オブジェクトを渡してモデルを開始し、最初から入力形状がわかるようにする必要があります。

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

`Input`オブジェクトはレイヤーではないため、`model.layers`の一部として表示されないことに注意してください。

In [ ]:
model.layers

簡単な代替手段として、最初のレイヤーに`input_shape`引数を渡すこともできます。

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

このような事前定義された入力形状で構築されたモデルは、常に (データが確認される前でも) 重みを持ち、常に定義された出力形状をもっています。

一般的に、Sequential モデルの入力形状がわかっている場合は、常に事前に指定しておくことをお勧めします。

## 一般的なデバッグワークフロー：`add()` + `summary()`

新しい Sequential アーキテクチャを構築する場合、`add()`を使用して段階的にレイヤーを積み重ね、モデルの概要を頻繁に出力することをお勧めします。これにより、`Conv2D`レイヤーと`MaxPooling2D`レイヤーのスタックが画像特徴量マップをどのようにダウンサンプリングしているかを監視できます。

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

これは、非常に実用的なワークフローです。


## モデルの準備ができたら

モデルアーキテクチャの準備ができたら、以下を行います。

- モデルをトレーニング、評価し、推論を実行します。[「トレーニングと組み込みループを使用した評価のガイド」](https://www.tensorflow.org/guide/keras/train_and_evaluate/)をご覧ください。
- モデルをディスクに保存して復元します。[「シリアル化と保存のガイド」](https://www.tensorflow.org/guide/keras/save_and_serialize/)をご覧ください。
- 複数の GPU を活用してモデルを迅速にトレーニングします。[「マルチ GPU と分散トレーニングのガイド」](https://keras.io/guides/distributed_training/)をご覧ください。

## Sequential モデルによる特徴量の抽出

Sequential モデルが構築されると、[Functional API モデル](https://www.tensorflow.org/guide/keras/functional/)のように動作します。つまり、すべてのレイヤーに`input`および`output`属性があります。これらの属性を使用すると、Sequential モデルのすべての中間層の出力を抽出するモデルを迅速に作成したりできます。

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

以下の例では、1 つのレイヤーからのみ特徴量を抽出します。

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

## Sequential モデルによる転移学習

転移学習では、モデルの最下層を凍結し、最上層のみをトレーニングします。転移学習の詳細については、[転移学習のガイド](https://www.tensorflow.org/guide/keras/transfer_learning/)を参照してください。

以下は、Sequential モデルを含む 2 つの一般的な転移学習の例です。

まず、Sequential モデルで、最後のレイヤーを除くすべてのレイヤーを凍結するとします。この場合、`model.layers` を繰り返し処理し、最後のレイヤーを除く各レイヤーに `layer.trainable = False` を設定します。 以下に例を示します。

```python
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)
```

もう 1 つの一般的な例は、Sequential モデルを使用して、事前にトレーニングされたモデルといくつかの新しく初期化された分類レイヤーをスタックすることです。以下に例を示します。

```python
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# Compile &amp; train
model.compile(...)
model.fit(...)
```

転移学習を行う場合、おそらくこれら 2 つのパターンを頻繁に使用されていることに気付くでしょう。

Sequential モデルについての説明は以上です。

Keras でのモデル構築の詳細については、以下を参照してください。

- [ Functional API のガイド](https://www.tensorflow.org/guide/keras/functional/)
- [サブクラス化でレイヤとモデルをゼロから作成する方法](https://www.tensorflow.org/guide/keras/custom_layers_and_models/)